In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import ariel

In [ ]:
X = ariel.read_spectra()
X.shape, X.dtype

In [ ]:
quartiles = ariel.read_quartiles_table()
quartiles.shape, quartiles.dtype

In [ ]:
Y = torch.from_numpy(quartiles[1]).float()
Y.shape, Y.dtype

In [ ]:
# TODO out-of-distribution split?
ids = torch.arange(ariel.N)
ids_train, ids_valid = train_test_split(ids, train_size=0.8, random_state=36)
idx_train = torch.zeros_like(ids, dtype=torch.bool)
idx_train[ids_train] = True
idx_valid = ~idx_train
X_train, X_valid = X[idx_train], X[idx_valid]
Y_train, Y_valid = Y[idx_train], Y[idx_valid]
quartiles_train, quartiles_valid = quartiles[:, idx_train], quartiles[:, idx_valid]
ids_train.shape, ids_valid.shape

In [ ]:
X_train_mean, X_train_std = X_train.mean(), X_train.std()
X_train = ariel.standardise(X_train, X_train_mean, X_train_std)
X_valid = ariel.standardise(X_valid, X_train_mean, X_train_std)

Y_train_mean, Y_train_std = Y_train.mean(dim=0), Y_train.mean(dim=0)
Y_train = ariel.standardise(Y_train, Y_train_mean, Y_train_std)
Y_valid = ariel.standardise(Y_valid, Y_train_mean, Y_train_std)

In [ ]:
X_train, X_valid = X_train.cuda(), X_valid.cuda()
Y_train, Y_valid = Y_train.cuda(), Y_valid.cuda()

In [ ]:
class MCCNN(nn.Module):
    def __init__(self, dropout_probability):
        super(MCCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 32, 3),
            nn.ReLU(),
            nn.Conv1d(32, 32, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 3),
            nn.ReLU(),
            nn.Conv1d(64, 64, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3),
            nn.ReLU(),
            nn.Conv1d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(384, 256),
            nn.Dropout(dropout_probability),
            nn.Linear(256, 256),
            nn.Dropout(dropout_probability),
            nn.Linear(256, ariel.N_TARGETS))

    def forward(self, X):
        return self.model(X)
    
    def loss(self, Y_pred, Y):
        return F.mse_loss(Y_pred, Y)

model = MCCNN(dropout_probability=0.1).cuda()

In [ ]:
from importlib import reload
reload(ariel)

trainloader = DataLoader(TensorDataset(X_train, Y_train), batch_size=256, shuffle=True)
ariel.train(
    model,
    trainloader,
    (X_valid, quartiles_valid),
    Y_train_mean, Y_train_std)